In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans


In [3]:
enel_sp = gpd.read_file("SP_BDGD_ENEL.shp") #conj no bdgd
# Lista para armazenar cada GeoDataFrame
frames = []

# Loop para ler cada arquivo GeoJSON e adicioná-lo à lista de frames
for i in range(1, 12):
    file_name = f"ucbt_{i:02d}_filtrada.geojson"
    gdf = gpd.read_file(file_name)
    frames.append(gdf)

# Concatenar todos os GeoDataFrames na lista em um único GeoDataFrame
ucbt = pd.concat(frames, ignore_index=True)

ucmt = gpd.read_file(r"H:\Meu Drive\Mestrado\ENEL\BDGD_ENEL_2023\UCMT.shp")
ucat = gpd.read_file(r"H:\Meu Drive\Mestrado\ENEL\BDGD_ENEL_2023\UCAT.shp")
ugbt = gpd.read_file(r"H:\Meu Drive\Mestrado\ENEL\BDGD_ENEL_2023\UGBT.shp")
ugmt = gpd.read_file(r"H:\Meu Drive\Mestrado\ENEL\BDGD_ENEL_2023\UGMT.shp")
ugat = gpd.read_file(r"H:\Meu Drive\Mestrado\ENEL\BDGD_ENEL_2023\UGAT.shp")

In [2]:
def create_map_point(final_gdf, entidade, cmapa="YlOrRd", high=False):
    final_gdf_base = final_gdf.copy() # Criar uma cópia do DataFrame
    meses = ["Janeiro", "Fevereiro", "Março", "Abril", "Maio", "Junho", "Julho", "Agosto", "Setembro", "Outubro", "Novembro", "Dezembro"]
    if high == True: #caso seja UCAT
        energy = ["ENE_01", "ENE_02", "ENE_03", "ENE_04", "ENE_05", "ENE_06", "ENE_07", "ENE_08", "ENE_09", "ENE_10", "ENE_11", "ENE_12"]
        for i in range(1, 13):
            final_gdf_base[energy[i-1]] = final_gdf_base[f"ENE_P_{i:02d}"] + final_gdf_base[f"ENE_F_{i:02d}"]
    for i in range(1, 13): # Demais entidades vem direto para cá
        final_gdf_men = final_gdf_base[(final_gdf_base[f"ENE_{i:02d}"] > 0)&(final_gdf_base[f"ENE_{i:02d}"] < 5000)].copy() # Fazer cópia aqui para evitar SettingWithCopyWarning
        optimal_clusters = 4

        kmeans = KMeans(n_clusters=optimal_clusters, n_init=10, random_state=0).fit(final_gdf_men[[f"ENE_{i:02d}"]])
        final_gdf_men['cluster'] = kmeans.labels_

        fig, axes = plt.subplots(2, 2, figsize=(10, 10))
        fig.suptitle(f"Consumo das {entidade}'s do mês de {meses[i-1]}", fontsize=16) # Título geral

        for j, ax in enumerate(axes.flatten()):
            enel_sp.plot(ax=ax, color="white", edgecolor="black")
            cluster_data = final_gdf_men[final_gdf_men['cluster'] == j]
            min_consumo = cluster_data[f"ENE_{i:02d}"].min()
            max_consumo = cluster_data[f"ENE_{i:02d}"].max()
            cluster_data.plot(ax=ax, column=f"ENE_{i:02d}", cmap=cmapa, markersize=8, linewidth=0.5, edgecolor="black", legend=True)
            ax.set_title(f'Consumo de {np.round(min_consumo, 2)} a {np.round(max_consumo, 2)} kWh') # Título de cada cluster
            ax.set_xlabel('Longitude')
            ax.set_ylabel('Latitude')

        plt.tight_layout(rect=[0, 0, 1, 0.96]) # Ajuste para não cortar o título
        plt.savefig(f"{entidade}_clusters_with_coordinates_{i:02d}.png", dpi=600)
        plt.close(fig)

In [4]:
create_map_point(ucbt, "UCBT")

KeyboardInterrupt: 

Error in callback <function flush_figures at 0x000001CAEE888220> (for post_execute):


KeyboardInterrupt: 

In [ ]:
#Específico para UGAT (pq só tem 3 pontos)
for i in range(1, 13):
    ugat[energy[i-1]] = ugat[f"ENE_P_{i:02d}"] + ugat[f"ENE_F_{i:02d}"]
meses = ["Janeiro", "Fevereiro", "Março", "Abril", "Maio", "Junho", "Julho", "Agosto", "Setembro", "Outubro", "Novembro", "Dezembro"]
energy = ["ENE_01", "ENE_02", "ENE_03", "ENE_04", "ENE_05", "ENE_06", "ENE_07", "ENE_08", "ENE_09", "ENE_10", "ENE_11", "ENE_12"]
for i in range(1, 13):
    fig, ax = plt.subplots(1, 1)
    fig.suptitle(f"Unidades UGAT do mês de {meses[i-1]}")
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')
    enel_sp.plot(ax=ax, color='white', edgecolor="black", legend=True) 
    ugat.plot(ax=ax, column=energy[i-1], cmap="YlGn", markersize=25, linewidth=0.7, edgecolor="black", legend=True)
    plt.savefig(f"UGAT_with_coordinates_{i:02d}.png", dpi=600)